In [2]:
!pip install transformers==4.10.1 --quiet

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 


In [3]:
!pip install pytorch_lightning

     |████████████████████████████████| 527 kB 5.4 MB/s 
     |████████████████████████████████| 952 kB 36.0 MB/s 
     |████████████████████████████████| 133 kB 40.2 MB/s 
     |████████████████████████████████| 829 kB 24.7 MB/s 
     |████████████████████████████████| 396 kB 33.6 MB/s 
     |████████████████████████████████| 1.1 MB 34.3 MB/s 
     |████████████████████████████████| 144 kB 31.3 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 35.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=7954ff9f06214a83936e1e6a10fb5c4db8116434bbdf905f467237cad3ca813c
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: f

In [4]:
!pip install "torchmetrics<0.7"

     |████████████████████████████████| 332 kB 5.3 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.7.0
    Uninstalling torchmetrics-0.7.0:
      Successfully uninstalled torchmetrics-0.7.0


In [5]:
#testing gpu prescence
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
import torch
torch.cuda.empty_cache()

In [7]:

#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
from tqdm.auto import tqdm

#pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel as BM, AdamW as Adam

#pytorch scheduler
from torch.optim.lr_scheduler import ExponentialLR

#pytorch lightning - lots of features like model checkpoints, logging, metrics, etc
import transformers
import pytorch_lightning as pl
#from torchmetrics import Accuracy
from torchmetrics.functional import auroc 
#going to create a confusion matrix
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [75]:
#IMP Params
EPOCHS = 4
BATCH_SIZE = 16
MAX_TOKEN_NUM = 256

In [76]:
#creating a training and testing dataset
labeledFile = open('news_blobs_labeled.json')
testingFile = open('news_blobs_test.json')
#loading data from file
labeledData = json.load(labeledFile)
testingData = json.load(testingFile)
labeledSize = len(labeledData)
testingSize = len(testingData)
print("Labeled data shape:", len(labeledData))
print("Testing data shape: ", len(testingData))


Labeled data shape: 738
Testing data shape:  81


In [77]:
#conversion into dataframes - training + validation
tdLabels = pd.DataFrame([labeledData[i]['labels'] for i in range(labeledSize)])
tdText= pd.DataFrame([{"text": labeledData[i]['text']} for i in range(labeledSize)])
totaldf = pd.concat([tdText, tdLabels], axis=1)
totaldf.head()

,text,REGULATORY,CLINICAL_TRIAL,COLLAB,FINANCING,PRESENTATION,OTHER
0,Cytokinetics to Participate in Upcoming Invest...,0,0,0,0,1,0
1,"Immunic, Inc. Reports Positive Top-line Data f...",0,1,0,0,0,0
2,FDA Accepts and Grants Priority Review of Vand...,1,0,0,0,0,0
3,Members Of The COVID R&D Alliance And Quantum ...,0,1,0,0,0,0
4,Lannett Announces Launch Of FDA Approved Levot...,0,0,0,0,0,1


In [78]:
COLUMNS = totaldf.columns.tolist()[1:]
COLUMNS

['REGULATORY',
 'CLINICAL_TRIAL',
 'COLLAB',
 'FINANCING',
 'PRESENTATION',
 'OTHER']

In [79]:
#use of validation data will help with accuracy prediction 
from sklearn.model_selection import train_test_split
traindf, valdf = train_test_split(totaldf, test_size=0.1)
traindf.shape, valdf.shape

((664, 7), (74, 7))

In [80]:
#what is the distribution of the labeled data given
samples = dict()
for col in COLUMNS:
  samples[col] = sum(totaldf[col])
samples

{'CLINICAL_TRIAL': 128,
 'COLLAB': 43,
 'FINANCING': 50,
 'OTHER': 415,
 'PRESENTATION': 46,
 'REGULATORY': 79}

In [81]:
#we may want pre-process the training dataset to counter the imbalance 
other_df = traindf[traindf["OTHER"] == 1]
other_df.head()

,text,REGULATORY,CLINICAL_TRIAL,COLLAB,FINANCING,PRESENTATION,OTHER
607,Free ME From Lung Cancer Achieves Accreditatio...,0,0,0,0,0,1
434,New Martinsville Spero Health Clinic Brings Ad...,0,0,0,0,0,1
521,Eric Feinstein Joins Navitus Health Solutions ...,0,0,0,0,0,1
337,Repare Therapeutics Reports Second Quarter 202...,0,0,0,0,0,1
339,NMS Labs Appoints Dan Monahan as President and...,0,0,0,0,0,1


In [82]:
#reduce the number of clinical trial alone ones
SELECT_COLUMNS = ['REGULATORY', 'COLLAB', "FINANCING", "PRESENTATION"]
clinic_df = traindf[traindf['CLINICAL_TRIAL'] == 1]
clinic_df = clinic_df[clinic_df[SELECT_COLUMNS ].sum(axis = 1) == 0]
clinic_df.head()

,text,REGULATORY,CLINICAL_TRIAL,COLLAB,FINANCING,PRESENTATION,OTHER
195,Oncopeptides initiates the first study with me...,0,1,0,0,0,0
401,American Journal of Psychiatry Publishes Data ...,0,1,0,0,0,0
402,Results of Axcella’s AXA1665-001 Clinical Stud...,0,1,0,0,0,0
271,Vaccibody AS and Nektar Therapeutics Announce ...,0,1,0,0,0,0
462,Deciphera Announces Publication of QINLOCK™ (r...,0,1,0,0,0,0


In [83]:
#training samples without the other label
non_other_df = traindf[traindf[SELECT_COLUMNS ].sum(axis = 1) > 0]
non_other_df.head(40)

,text,REGULATORY,CLINICAL_TRIAL,COLLAB,FINANCING,PRESENTATION,OTHER
657,Exicure Announces a Virtual KOL Meeting to Dis...,0,0,0,0,1,0
550,After Several Months of Providing Requested In...,1,1,0,0,0,0
687,Calyxt to Present at Upcoming September Invest...,0,0,0,0,1,0
594,MC2 Therapeutics Announces Collaboration Agree...,0,0,1,0,0,0
706,REGENXBIO Announces FDA Clearance of IND for P...,1,1,0,0,0,0
314,Australian Therapeutic Goods Administration Ap...,1,0,0,0,0,0
110,FDA Accepts Application for Xolair (omalizumab...,1,0,0,0,0,0
278,Strata Oncology Announces Partnership to Broad...,0,0,1,0,0,0
443,Oita Chuo Pacific Management Comments on Sanof...,0,0,1,0,0,0
79,Frequency Therapeutics Provides Business Updat...,0,1,0,1,0,0


In [84]:
#sample accordingly to avoid biased training data
print(other_df.shape, non_other_df.shape)
minSize = min(other_df.shape[0], non_other_df.shape[0])
traindf = pd.concat([non_other_df.sample(minSize), clinic_df.sample(50), other_df.sample(minSize)])

(374, 7) (189, 7)


In [134]:
#removing the "other" column
if "OTHER" in traindf.columns:
  traindf = traindf.drop("OTHER", 1)
if "OTHER" in valdf.columns:
  valdf = valdf.drop("OTHER", 1)
traindf.head()

,text,REGULATORY,CLINICAL_TRIAL,COLLAB,FINANCING,PRESENTATION
527,Axcella Announces Upcoming Later-Breaker Prese...,0,1,0,0,1
218,Apnimed to Present at Oppenheimer’s Private Li...,0,0,0,0,1
644,Silicon Valley Startup Exo Secures $40M in Ser...,0,0,0,1,0
131,Horizon Therapeutics plc Announces Closing of ...,0,0,0,1,0
690,Can-Fite Receives Notification of Patent Grant...,1,1,0,0,0


In [110]:
traindf.head(50)

,text,REGULATORY,CLINICAL_TRIAL,COLLAB,FINANCING,PRESENTATION
527,Axcella Announces Upcoming Later-Breaker Prese...,0,1,0,0,1
218,Apnimed to Present at Oppenheimer’s Private Li...,0,0,0,0,1
644,Silicon Valley Startup Exo Secures $40M in Ser...,0,0,0,1,0
131,Horizon Therapeutics plc Announces Closing of ...,0,0,0,1,0
690,Can-Fite Receives Notification of Patent Grant...,1,1,0,0,0
274,Innovent and Eli Lilly Announce Acceptance of ...,1,0,0,0,0
397,"Izabela Podgorski, Ph.D., researcher at Karman...",0,0,0,1,0
586,Oragenics Signs Process Development and Manufa...,0,0,1,0,0
542,Sage Therapeutics to Host Sage FutureCast Webc...,0,0,0,0,1
151,THREAD Accelerates Expansion to Advance Global...,0,0,0,1,0


In [86]:
#labels with "other"
if "OTHER" in COLUMNS:
    COLUMNS.remove("OTHER")

In [87]:
#conversion into dataframes - testing  
testdf = pd.DataFrame([{"text": testingData[i]['text']} for i in range(testingSize)])
testdf.head()

,text
0,RLF-100 (aviptadil) clinical trial showed rap...
1,Bio-Techne And Kantaro Biosciences To Launch A...
2,AIVITA Biomedical Publishes Review of GM-CSF H...
3,Biohaven Announces Enrollment Of First Patient...
4,Mirum Pharmaceuticals Reports Inducement Grant...


In [88]:
#initial look into the data
print('Research text sample:', traindf['text'].iloc[0])

Research text sample: Axcella Announces Upcoming Later-Breaker Presentation at The Digital International Liver Congress 2020. Axcella (Nasdaq: AXLA), a clinical-stage biotechnology company focused on leveraging endogenous metabolic modulators (EMMs) to pioneer a new approach for treating complex diseases and improving health, today announced an upcoming late-breaker poster presentation at The Digital International Liver Congress™ 2020 (EASL), which is being held virtually from August 27 to 29. This presentation will discuss key data from AXA1125-003, a placebo-controlled, randomized, multi-arm clinical study assessing the impact of AXA1125 and AXA1957 on safety, tolerability and effects on structures and functions of the liver in 102 adult subjects with non-alcoholic fatty liver disease (NAFLD).


In [89]:
#clean titles
for i in range(traindf.shape[0]):
  traindf['text'].iloc[i].strip().lower()
for i in range(valdf.shape[0]):
  valdf['text'].iloc[i].strip().lower()
for i in range(testingSize):
  testdf['text'].iloc[i].strip().lower()

In [115]:
#before processing data, create tokenizer instance
'''
BERT is a transformers model pretrained on a large corpus of English data in a self-supervised fashion.
'''
BERT_MODEL_NAME_0 = 'bert-base-cased'
BERT_MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [116]:
#creating a pytorch dataset class
class MedicalDataset(Dataset):
  def __init__(self, data,  tokenizer):
    super().__init__()
    self.tokenizer = tokenizer
    self.data = data
    self.dataLen = len(self.data)

  def __len__(self):
    return self.dataLen

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    medical_text = data_row['text']
    labels = data_row[COLUMNS]

    #creating an encoding 
    encoding = self.tokenizer.encode_plus(
      medical_text,
      add_special_tokens=True,
      max_length=MAX_TOKEN_NUM,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return dict(
      medical_text=medical_text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels=torch.FloatTensor(labels)
    )

In [117]:
class MedicalTextDataModule(pl.LightningDataModule):
  #three datasets - training, testing, validation dataset
  def __init__(self, train_df, val_df, test_df, tokenizer):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.val_df = val_df
    self.tokenizer = tokenizer

  #datasets setup - training, validation, testing  
  def setup(self, stage=None):
    self.train_dataset = MedicalDataset(
      self.train_df,
      self.tokenizer
    )
    self.val_dataset = MedicalDataset(
        self.val_df, 
        self.tokenizer)
    
    self.test_dataset = MedicalDataset(
      self.test_df,
      self.tokenizer
    )

  #The num_workers attribute tells the data loader instance how many sub-processes to use for data loading.
  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size= BATCH_SIZE,
      shuffle=True,
      num_workers=2)
    
  def val_dataloader(self):
    return DataLoader(
      self.val_dataset,
      batch_size= BATCH_SIZE,
      shuffle=True,
      num_workers=2)

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=BATCH_SIZE,
      num_workers=2)

In [137]:
#load a pre-trained BERT model - transfer learning
bert_model = BM.from_pretrained(BERT_MODEL_NAME, return_dict=True)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [138]:
#create a data module with all datasets in place
med_data_module = MedicalTextDataModule(traindf, valdf, valdf, tokenizer)

In [139]:
class MedicalTextLabelClassifier(pl.LightningModule):

  def __init__(self, bert_model, n_classes, criterion, n_training_steps=None):
    super().__init__()
    #pre-trained bert_model
    self.bert = bert_model
    #classifier linear model
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    #how many times step is called
    self.n_training_steps = n_training_steps
    self.criterion = criterion

  #the forward function enables us to define how the model goes from input to output
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    #last layer to a classifier output
    output = self.classifier(output.pooler_output)
    #our activation function
    output = torch.sigmoid(output)    
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    #attention masks - binary masks to hide the padded indices
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    if batch_idx % 5000:
      print({"loss": loss})
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    return loss

  def training_epoch_end(self, outputs):
    #collecting the labels and predictions from the output
    labels = []
    predictions = []
    for output in outputs:
      for output_labels in output["labels"].detach().cpu():
        labels.append(output_labels)
      for output_predictions in output["predictions"].detach().cpu():
        predictions.append(output_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)


  def configure_optimizers(self):

    optimizer = Adam(self.parameters(), lr=1e-5)

    #the scheduler enables us to adjust the learning rate
    scheduler = ExponentialLR(optimizer, gamma=0.9)

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [140]:
#criterion - measuring the Binary Cross Entropy between the target and the input probabilities
criterion = nn.BCELoss()

In [141]:
#training
#calculating number of train steps
total_training_steps = BATCH_SIZE * EPOCHS

model = MedicalTextLabelClassifier(
    bert_model,
  n_classes=len(COLUMNS),
  criterion = criterion,
  n_training_steps=total_training_steps 
)

#lightning module training configs
trainer = pl.Trainer(
  logger= None,
  checkpoint_callback= None,
  callbacks= None,
  gpus=1,
  max_epochs=EPOCHS
)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [142]:
#clear memory
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
#fitting 
trainer.fit(model, med_data_module)

In [144]:
#evaluation mode
model.eval()
model.freeze()

In [145]:
device = torch.device('cpu')
trained_model = model.to(device)
#evaluation
val_dataset = MedicalDataset(valdf, tokenizer)

predictions = []
labels = []

#wrap the iterable with tqdm
for item in tqdm(val_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

  0%|          | 0/74 [00:00<?, ?it/s]

In [156]:
CUTOFF = 0.5
#fscore, precision, recall, etc
y_pred = predictions.numpy()

#true labels, vs predictions - assigned an upper and lower bound
y_true = labels.numpy()
y_pred = np.where(y_pred >= CUTOFF, 1, 0)


In [ ]:
#confusion matrices
cm = multilabel_confusion_matrix(y_true, y_pred)
cm

In [148]:
#accuracy
accDict = dict()
for i, label in enumerate(COLUMNS):
  row = cm[i]
  tp = row[0][0]
  tn = row[1][1]
  fp = row[0][1]
  fn = row[1][0]
  accDict[label] = ((tp+tn)/(tp+tn+fp+fn))
accDict
  

{'CLINICAL_TRIAL': 0.8513513513513513,
 'COLLAB': 0.9459459459459459,
 'FINANCING': 0.9324324324324325,
 'PRESENTATION': 0.918918918918919,
 'REGULATORY': 0.9054054054054054}

In [149]:
print("AUROC per label")
for i, name in enumerate(COLUMNS):
  tag_auroc = auroc(predictions[:, i], labels[:, i], pos_label=1)
  print(f"{name}: {tag_auroc}")

AUROC per label
REGULATORY: 0.8422175049781799
CLINICAL_TRIAL: 0.4559885263442993
COLLAB: 0.550000011920929
FINANCING: 0.5478261113166809
PRESENTATION: 0.9240196347236633


In [150]:
def create_test_encodings(text):
  encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_TOKEN_NUM,
        return_token_type_ids=False,
        padding="max_length",
        return_attention_mask=True,
        return_tensors='pt',
      )
  return encoding

In [153]:
#testing 
def run_tests(path):
    results = []
    resultsFilePath = path
    for i in range(testingSize):
      #extract the text of the test news blobs
      test_text = testdf.iloc[i]['text']
      #create an encoding
      encoding = create_test_encodings(test_text)
      #use our trained model
      _, test_prediction = trained_model(encoding["input_ids"], encoding["attention_mask"])
      
      test_prediction = test_prediction.flatten().numpy()
      predDict = {}
      for label, prediction in zip(COLUMNS, test_prediction):
        if prediction < CUTOFF - 0.1:
          predDict[label] = 0
        else:
          
          predDict[label] = 1

      sumVar = 0 
      for label in predDict:
          sumVar += predDict[label]
      if sumVar == 0:
        predDict["OTHER"] = 1
      else:
        predDict["OTHER"] = 0
      predDict['text'] = test_text
      results.append(predDict)

    resultsDF = pd.DataFrame(results)  
    resultsDF.to_csv(path, index=False)
    return 
    

In [154]:
run_tests("text_classification.csv")